# Train a neural network (velocity for ramps with gaussian noise)

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Set seeds

In [ ]:
np.random.seed(1)
tf.random.set_seed(1)

Set parameters

In [ ]:
# file with training input and output data: format is x,y
input_file = 'MAR2_negative_ramps.csv'

# names of input/output columns
inputs = ['mu', 'length', 'angle']
outputs = ['speed']

# these set the input/output dimensions of the network
input_size = len(inputs)
output_size = len(outputs)

Read data

In [ ]:
data = pd.read_csv(input_file)
data.columns = inputs + outputs

x = np.array(data[inputs])
y = np.array(data[outputs])

Normalizing preprocessing layer from training data

In [ ]:
normalizer = keras.layers.experimental.preprocessing.Normalization()
normalizer.adapt(x)

Base model

In [ ]:
input_shape = x.shape[1:]
full_model_input = keras.Input(shape=input_shape)
normalized_input = normalizer(full_model_input)
noise_layer_1 = keras.layers.GaussianNoise(0.025)(normalized_input[...,0],training=True)
noise_layer_2 = keras.layers.GaussianNoise(0.025)(normalized_input[...,1],training=True)
noise_layer_3 = keras.layers.GaussianNoise(0.025)(normalized_input[...,2],training=True)
reshape_layer_1 = keras.layers.Reshape((1,))(noise_layer_1)
reshape_layer_2 = keras.layers.Reshape((1,))(noise_layer_2)
reshape_layer_3 = keras.layers.Reshape((1,))(noise_layer_3)
recombined_input = keras.layers.Concatenate()([reshape_layer_1, reshape_layer_2, reshape_layer_3])
dense_layer_1 = keras.layers.Dense(6, activation='relu')(recombined_input)
#dense_layer_2 = keras.layers.Dense(3, activation='relu')(dense_layer_1)
full_model_output = keras.layers.Dense(output_size,activation='relu')(dense_layer_1)
full_model = keras.Model(full_model_input, full_model_output)
full_model.compile(loss='mse',optimizer='adam',metrics=['mse'])

Split data

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, shuffle= True)

Train model

In [ ]:
model_output = full_model.fit(x_train,y_train,epochs=1000,batch_size=5,verbose=0,validation_data=(x_val,y_val)) # check validation

View training

In [ ]:
plt.yscale('log')
plt.title('loss')
plt.plot(model_output.history['loss'], label='train')
plt.plot(model_output.history['val_loss'], label='validation')
plt.legend()
plt.figure()
plt.yscale('log')
plt.title('mse')
plt.plot(model_output.history['mse'], label='train')
plt.plot(model_output.history['val_mse'], label='validation')
plt.legend()
plt.figure()

View output

In [ ]:
# plot speed vs angle given mu, length
mu = 1 # set mu
length = 1 # set length
angles = angles = data['angle'].unique()
angle = np.linspace(min(angles),max(angles),50)

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if (np.isclose(i[0], mu) and np.isclose(i[1], length))] # x, y
# this is not generic enough...
pred_x = [[mu, length, a] for a in angle]
pred = full_model.predict(pred_x)

plt.scatter([i[2] for i in plot_x], [y[i] for i,v in enumerate(x) if (np.isclose(v[0], mu) and np.isclose(v[1], length))])
plt.scatter([i[2] for i in pred_x], pred)
plt.title('mu: %0.2f, length: %0.2f' % (mu, length))

In [ ]:
mu = 1 # set mu
length = 1 # set length
angles = angles = data['angle'].unique()
angle = np.linspace(min(angles),max(angles),50)

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
for i in range(0, 10):
    plot_x = [i for i in x if (np.isclose(i[0], mu) and np.isclose(i[1], length))] # x, y    # this is not generic enough...
    pred_x = [[mu, length, a] for a in angle]
    pred = full_model.predict(pred_x)
    plt.scatter([i[2] for i in pred_x], pred, c='orange', alpha='0.3')
plt.scatter([i[2] for i in plot_x], [y[i] for i,v in enumerate(x) if (np.isclose(v[0], mu) and np.isclose(v[1], length))])
plt.title('mu: %0.2f, length: %0.2f' % (mu, length))

Save model

In [ ]:
full_model.save('1_speed_ramp_network_with_noise.h5')

# Predictions

In [ ]:
# plot speed vs angle given mu, length
angle = np.linspace(-0.6,0.65,50)
mu = 1 # set mu
length = 1 # set length
runs = 10 # runs per prediction

# bug: mu = 0.009 is read as 0.0090..01
#plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
plot_x = [i for i in x if (np.isclose(i[0], mu) and np.isclose(i[1], length))] # x, y
# this is not generic enough...
pred_x = [[mu,length,a] for a in angle]
pred = []

plt.title('mu = %s, length = %s' % (mu, length))
plt.xlabel('angle (radians)')
plt.ylabel('speed (m/s)')
plt.scatter([i[2] for i in plot_x], [y[i] for i,v in enumerate(x) if np.isclose(v[0], mu) and np.isclose(v[1], length)], marker='o', label='training')

for i in pred_x:
    point_predictions = full_model.predict([i for run in range(0, runs)]) # faster than np.tile for small arrays
    point_predictions = np.array(point_predictions)
    variance = point_predictions.var(axis = 0, ddof = 1) # unbiased estimator
    mean = point_predictions.mean()
    if i[2] > 0: # if it is an up ramp
        pred.append(mean + 2 * (variance**0.5)) # overpredict
    else: # if it is a down ramp
        pred.append(mean - 2 * (variance**0.5)) # underpredict

plt.scatter([i[2] for i in pred_x], pred, label='predictions', c='orange')
plt.legend()

Make a lot of plots

In [ ]:
mus = data['mu'].unique()
lengths = data['length'].unique()
angles = data['angle'].unique()
a=0
# plot speed vs angle given mu, threshold
for mu in mus:
    for length in lengths:
        # bug: mu = 0.009 is read as 0.0090..01
        #plot_values = [i for i in x if i[0] == mu and i[2] == thresh] # x, y
        plot_x = [i for i in x if (np.isclose(i[0], mu) and np.isclose(i[1], length))] # x, y
        pred_x = [[mu,length,angle] for angle in np.linspace(angles.min(),angles.max(),30)]
        pred = full_model.predict(pred_x)
        #plt.plot([i[1] for i in plot_values], [y[i] for i,v in enumerate(x) if v[0] == mu and v[2] == thresh])
        fig = plt.figure()
        plt.title('mu: %.3f, length: %.2f' % (mu, length))
        plt.scatter([i[2] for i in plot_x], [y[i] for i,v in enumerate(x) if (np.isclose(v[0], mu) and np.isclose(v[1], length))],
                label = 'training')
        plt.plot([i[2] for i in pred_x], pred, label = 'predicted', color='orange', marker='o')
        plt.legend()
        plt.savefig('plots/mu-%.3f_length-%.2f.png' % (mu,length))
        plt.close()